# Imports

In [1]:
import ee 
import numpy as np 
import webbrowser
import pandas as pd
import os 

#visualization
import matplotlib.pyplot as plt 
from matplotlib.colors import ListedColormap
import folium
import branca.colormap as cmp

#inspect source
import inspect

from PIL import Image
import IPython.display as disp
%matplotlib inline

In [2]:
cwd =os.path.abspath(os.getcwd())

cwd

'/kaggle/working'

# Authenticate Earth Engine

In [5]:
#geenrate a authentication token
# ee.Authenticate()

In [6]:
ee.Initialize()

# Helper Functions

In [7]:
def add_ee_layer(self, ee_image_object, vis_params, name,opacity=0.90):
    '''function to add GEE layer to folium,
    Source : GEE'''
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
          tiles=map_id_dict['tile_fetcher'].url_format,
          attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
          name=name,
          opacity=opacity,
          overlay=True,
          control=True
      ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

# Getting Image Boundaries and DEM Image

In [8]:
#the dataset with boundaries info
bounds= ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")

#get geometry for State of Maharashtra
mh = bounds.filter(ee.Filter.eq('ADM1_NAME', 'Maharashtra')).geometry()



#get a clipped image of Digital Elevation
dem = ee.Image("NASA/NASADEM_HGT/001").clip(mh)

# srtm = ee.Image('CGIAR/SRTM90_V4').clip(mh)

In [9]:
#select the elevation band
elev = dem.select('elevation')

In [10]:
#slope: Slope is the steepness or the degree of incline of a surface. 
slope= ee.Terrain.slope(dem.select('elevation'))

#aspect : Aspect is the orientation of slope, measured clockwise in degrees from 0 to 360, 
# where 0 is north-facing, 90 is east-facing, 180 is south-facing, and 270 is west-facing.
aspect = ee.Terrain.aspect(dem.select('elevation'))

In [11]:
#find tallest region

highest_point=elev.reduceRegion(**{
                        'reducer':ee.Reducer.max(),
                        'geometry':mh,
                        'scale':30,
                        'maxPixels':1e10}).get('elevation').getInfo()

#find lowest point
lowest_point=elev.reduceRegion(**{
                        'reducer':ee.Reducer.min(),
                        'geometry':mh,
                        'scale':30,
                        'maxPixels':1e10}).get('elevation').getInfo()


#note that the highest point here may not correspond to actual highest point as our scale is 30(i.e each pixel in image is atleast 30*30m)
print(f'Highest point found is {highest_point} meters')
print(f'Lowest point found is {lowest_point} meters')


Highest point found is 1564 meters
Lowest point found is -100 meters


# Visualize Terrain:


**This Map shows the topography of the region. The higher regions are marked with a darker hue, while the lower regions are marked with lighter colors**

In [12]:
# #loaction to initialize at :
loc= (19.663280, 75.300293)


# map
map_1 = folium.Map(location=loc, zoom_start=8)



# palette = ListedColormap(["white", "tan", "springgreen", "darkgreen"])
color_list=['green','yellow','orange','brown','black']

viz1 = {'bands':['elevation'],
        'min': -100, 'max': 1500, 
        'palette': color_list}

#add color legend
cbar = cmp.StepColormap(
 color_list,
 vmin=-100, vmax=1500,
 caption='Elevation'
).add_to(map_1)


# Add the image layer to the map and display it.
map_1.add_ee_layer(dem,viz1,'DEM')
map_1.add_child(folium.LayerControl())


print('Digital Elevation Model')


#save map 
map_1.save('DEM.html')

disp.IFrame('./DEM.html',width=1100, height=500)

Digital Elevation Model


# Showing low lying regions

**The darker hues show the presence of river valleys,while the lighter ones show tall ground. On the western region, the darker regions show coastal lowlands along with river valleys.**

In [13]:

m_5 =folium.Map(location=loc, zoom_start=8)

color_list2=['green','yellow','orange','brown','black'][::-1]
viz5={'min': -100, 'max': 1300, 
        'palette': color_list2}


#add color legend
cbar = cmp.StepColormap(
 color_list2,
 vmin=-100, vmax=1300,
 caption='Elevation'
).add_to(m_5)


# Add the image layer to the map and display it.
m_5.add_ee_layer(elev,viz5,'Valleys')
m_5.add_child(folium.LayerControl())


print('Downward sloping regions')


#save map 
m_5.save('valleys.html')
disp.IFrame('./valleys.html',width=1100, height=500)

Downward sloping regions


# Visualizing slope


**The Whiter shade shows high degree of slope, while the darker region shows flat regions**

In [14]:
#loaction to initialize at :
loc= (19.663280, 75.300293)


# map
map2 = folium.Map(location=loc, zoom_start=8)

cl2 = ['black','grey','white']


viz1 = {'min':0, 'max': 75, 
        'palette': cl2}



map2.add_ee_layer(slope,viz1,'Slope',opacity=0.97)
map2.add_child(folium.LayerControl())



print('Slopes')

#save map 
map2.save('slope.html')
disp.IFrame('./slope.html',width=1100, height=500)

Slopes


# Find regions to trek (regions with slope  between 20- 50 degrees and elevation greater than 800 m)

In [15]:
# # elev1: areas>800m, 
# # slope1 greater than 20 deg and less than 50 degrees

slope1= slope.updateMask(slope.gte(20).And(slope.lte(50)))

# #mask elevation > 800m
ele_mask = elev.gte(800)

# #udate mask
elev1= elev.updateMask(ele_mask)



m_2 =folium.Map(location=loc, zoom_start=8,tiles='cartodb positron')



clist2 = ['orange','brown','black']

#elev viz
viz_elev={'min': 800, 'max': 1500, 
        'palette': clist2}

#add color legend
cbar = cmp.StepColormap(
 clist2,
 vmin=800, vmax=1500,
 caption='Elevation'
).add_to(m_2)


#slope_viz
slope_colors=['lightgreen','green','darkgreen']
viz_slp={'min': 20, 'max': 50, 
        'palette': slope_colors}
cbar = cmp.StepColormap(
 slope_colors,
 vmin=20, vmax=50,
 caption='slope'
).add_to(m_2)





#add elevation layer
m_2.add_ee_layer(elev1,viz_elev,'Higher regions')

#add slope layer
m_2.add_ee_layer(slope1,viz_slp,'Slope')

m_2.add_child(folium.LayerControl())


print('Probable areas for treks')

#save map 
m_2.save('trek.html')
disp.IFrame('./trek.html',width=1100, height=500)

Probable areas for treks


# gentler slopes (height > 600 m and <800m) and (5<slope < 30)

In [16]:

m_2 =folium.Map(location=loc, zoom_start=8,tiles='cartodb positron')


# # elev1: areas>800m, 
# # slope1 greater than 5 deg and less than 30 degrees

slope2= slope.updateMask(slope.gte(5).And(slope.lte(30)))


# #mask elevation > 800m
ele_mask = elev.lt(800).And(elev.gte(600))

# #udate mask
elev2= elev.updateMask(ele_mask)

#elev viz
viz_elev={'min': 800, 'max': 1500, 
        'palette': clist2}

#add color legend
cbar = cmp.StepColormap(
 clist2,
 vmin=600, vmax=800,
 caption='Elevation'
).add_to(m_2)


#slope_viz
viz_slp={'min': 5, 'max': 30, 
        'palette': slope_colors}
cbar = cmp.StepColormap(
 slope_colors,
 vmin=5, vmax=30,
 caption='slope'
).add_to(m_2)




#add elevation layer
m_2.add_ee_layer(elev2,viz_elev,'Higher regions')

#add slope layer
m_2.add_ee_layer(slope2,viz_slp,'Slope')

m_2.add_child(folium.LayerControl())


print('Probable areas for hikes')

#save map 
m_2.save('hike.html')
disp.IFrame('./hike.html',width=1100, height=500)

Probable areas for hikes
